In [1]:
import pandas as pd
import numpy as np
import os
import glob

# 1. CẤU HÌNH
pd.set_option('display.float_format', lambda x: '%.2f' % x)

BASE_DIR = '..'
CITY_FOLDER = 'berlin'  
PROCESSED_DIR = os.path.join(BASE_DIR, 'processed', CITY_FOLDER)
KPI_OUTPUT_DIR = os.path.join(BASE_DIR, 'processed', CITY_FOLDER) 

print(f"--- BẮT ĐẦU TÍNH TOÁN KPI FULL CHO: {CITY_FOLDER.upper()} ---")

# 2. HÀM TÍNH TOÁN KPI CHO TỪNG SNAPSHOT
def calculate_snapshot_kpi(snapshot_path):
    snapshot_name = os.path.basename(snapshot_path)
    print(f"\n>> Đang xử lý snapshot: {snapshot_name}")
    
    # --- Load Data ---
    try:
        df_list = pd.read_csv(os.path.join(snapshot_path, 'listings_processed.csv'), low_memory=False)
        df_cal = pd.read_csv(os.path.join(snapshot_path, 'calendar_processed.csv'), low_memory=False)
        
        try:
            df_rev = pd.read_csv(os.path.join(snapshot_path, 'reviews_processed.csv'), low_memory=False)
            df_rev['date'] = pd.to_datetime(df_rev['date'])
        except FileNotFoundError:
            df_rev = pd.DataFrame(columns=['listing_id', 'date']) # Tạo bảng rỗng nếu thiếu
            
    except FileNotFoundError:
        print("   [!] Không tìm thấy file listing/calendar. Bỏ qua.")
        return None, None, None, None, None

    # --- Lọc dữ liệu sạch (QA Filter) ---
    # Chỉ lấy listing không bị flag lỗi giá hoặc lỗi toạ độ
    # (Nếu cột qa_flag không tồn tại thì coi như hợp lệ)
    if 'qa_flag_price_zero' in df_list.columns:
        df_list = df_list[df_list['qa_flag_price_zero'] == False]
    if 'qa_flag_out_of_city' in df_list.columns:
        df_list = df_list[df_list['qa_flag_out_of_city'] == False]
        
    valid_listings = df_list.copy()
    valid_ids = valid_listings['id'].unique()
    
    # Lọc calendar chỉ chứa các listing hợp lệ
    valid_cal = df_cal[df_cal['listing_id'].isin(valid_ids)].copy()

    # ==============================================================================
    # PHẦN 1: KPI TỔNG HỢP & ROOM TYPE
    # ==============================================================================
    total_supply = len(valid_listings)
    
    # Multi-hosts
    host_counts = valid_listings['host_id'].value_counts()
    multi_host_rate = (host_counts[host_counts > 1].sum() / total_supply) * 100 if total_supply > 0 else 0

    # Median Price & Avail
    median_price = valid_listings['price_numeric'].median()
    median_avail_90 = valid_listings['availability_90'].median()
    
    # Room Type Stats
    room_stats = valid_listings['room_type'].value_counts(normalize=True).reset_index()
    room_stats.columns = ['room_type', 'percentage']
    room_stats['percentage'] = room_stats['percentage'] * 100
    room_stats['snapshot_date'] = snapshot_name

    # Summary Row
    summary_row = {
        'snapshot_date': snapshot_name,
        'total_listings': total_supply,
        'multi_host_rate': multi_host_rate,
        'median_price': median_price,
        'median_avail_90': median_avail_90
    }

    # ==============================================================================
    # PHẦN 2: SEASONALITY (CALENDAR)
    # ==============================================================================
    valid_cal['date'] = pd.to_datetime(valid_cal['date'])
    valid_cal['is_booked'] = valid_cal['available'].apply(lambda x: 1 if x == 'f' else 0)
    
    daily_occupancy = valid_cal.groupby('date')['is_booked'].mean().reset_index()
    daily_occupancy.columns = ['date', 'occupancy_rate']
    daily_occupancy['snapshot_date'] = snapshot_name

    # ==============================================================================
    # PHẦN 3: NEIGHBOURHOOD 
    # ==============================================================================
    # Dùng cột neighbourhood_cleansed hoặc neighbourhood
    col_neigh = 'neighbourhood_cleansed' if 'neighbourhood_cleansed' in valid_listings.columns else 'neighbourhood'
    
    neigh_stats = valid_listings.groupby(col_neigh).agg({
        'id': 'count',
        'price_numeric': 'median'
    }).reset_index()
    neigh_stats.columns = ['neighbourhood', 'listing_count', 'median_price']
    neigh_stats['snapshot_date'] = snapshot_name

    # ==============================================================================
    # PHẦN 4: REVIEW TRENDS 
    # ==============================================================================
    # Resample theo tháng
    if not df_rev.empty:
        review_trends = df_rev.set_index('date').resample('M').size().reset_index(name='review_count')
        review_trends['snapshot_date'] = snapshot_name
        # Chỉ lấy dữ liệu review của 2 năm gần nhất so với snapshot để giảm nhiễu
        # (Tuỳ chọn: Nếu muốn lấy hết thì bỏ dòng lọc này)
    else:
        review_trends = pd.DataFrame()

    return summary_row, room_stats, daily_occupancy, neigh_stats, review_trends

# 3. CHẠY VÒNG LẶP & TỔNG HỢP
all_snapshots = glob.glob(os.path.join(PROCESSED_DIR, '*'))
summary_list = []
room_stats_list = []
occupancy_list = []
neigh_stats_list = []
review_stats_list = []

for folder in all_snapshots:
    if os.path.isdir(folder):
        res_sum, res_room, res_occ, res_neigh, res_rev = calculate_snapshot_kpi(folder)
        if res_sum:
            summary_list.append(res_sum)
            room_stats_list.append(res_room)
            occupancy_list.append(res_occ)
            neigh_stats_list.append(res_neigh)
            if not res_rev.empty: review_stats_list.append(res_rev)

# 4. GỘP & LƯU FILE
if summary_list:
    # 4.1 Summary General
    df_summary = pd.DataFrame(summary_list).sort_values('snapshot_date')
    df_summary.to_csv(os.path.join(KPI_OUTPUT_DIR, f'kpi_summary_general_{CITY_FOLDER}.csv'), index=False)
    
    # 4.2 Room Type
    if room_stats_list:
        pd.concat(room_stats_list).to_csv(os.path.join(KPI_OUTPUT_DIR, f'kpi_room_type_{CITY_FOLDER}.csv'), index=False)
    
    # 4.3 Seasonality
    if occupancy_list:
        pd.concat(occupancy_list).to_csv(os.path.join(KPI_OUTPUT_DIR, f'kpi_seasonality_{CITY_FOLDER}.csv'), index=False)

    # 4.4 Neighbourhood 
    if neigh_stats_list:
        pd.concat(neigh_stats_list).to_csv(os.path.join(KPI_OUTPUT_DIR, f'kpi_neighbourhood_{CITY_FOLDER}.csv'), index=False)

    # 4.5 Review Trends 
    if review_stats_list:
        # Gom nhóm lại lần nữa nếu có nhiều snapshot (để tránh trùng lặp nếu vẽ biểu đồ tổng)
        # Thường ta chỉ lấy trend từ snapshot mới nhất, hoặc gộp tất cả. 
        # Ở đây mình gộp tất cả để bạn dễ xử lý.
        pd.concat(review_stats_list).to_csv(os.path.join(KPI_OUTPUT_DIR, f'kpi_reviews_trend_{CITY_FOLDER}.csv'), index=False)

    print("\n✅ ĐÃ HOÀN TẤT! Đã tạo ra 5 file KPI:")
    print(f"   1. kpi_summary_general_{CITY_FOLDER}.csv")
    print(f"   2. kpi_room_type_{CITY_FOLDER}.csv")
    print(f"   3. kpi_seasonality_{CITY_FOLDER}.csv")
    print(f"   4. kpi_neighbourhood_{CITY_FOLDER}.csv")
    print(f"   5. kpi_reviews_trend_{CITY_FOLDER}.csv")

else:
    print("Không có dữ liệu để tính toán.")

--- BẮT ĐẦU TÍNH TOÁN KPI FULL CHO: BERLIN ---

>> Đang xử lý snapshot: 15 March, 2025


C:\Users\Admin\AppData\Local\Temp\ipykernel_6464\430018417.py:106: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  review_trends = df_rev.set_index('date').resample('M').size().reset_index(name='review_count')



>> Đang xử lý snapshot: 20 June, 2025


C:\Users\Admin\AppData\Local\Temp\ipykernel_6464\430018417.py:106: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  review_trends = df_rev.set_index('date').resample('M').size().reset_index(name='review_count')



>> Đang xử lý snapshot: 21 December, 2024


C:\Users\Admin\AppData\Local\Temp\ipykernel_6464\430018417.py:106: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  review_trends = df_rev.set_index('date').resample('M').size().reset_index(name='review_count')



>> Đang xử lý snapshot: 23 September, 2025


C:\Users\Admin\AppData\Local\Temp\ipykernel_6464\430018417.py:106: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  review_trends = df_rev.set_index('date').resample('M').size().reset_index(name='review_count')



✅ ĐÃ HOÀN TẤT! Đã tạo ra 5 file KPI:
   1. kpi_summary_general_berlin.csv
   2. kpi_room_type_berlin.csv
   3. kpi_seasonality_berlin.csv
   4. kpi_neighbourhood_berlin.csv
   5. kpi_reviews_trend_berlin.csv


C:\Users\Admin\AppData\Local\Temp\ipykernel_6464\430018417.py:145: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat(occupancy_list).to_csv(os.path.join(KPI_OUTPUT_DIR, f'kpi_seasonality_{CITY_FOLDER}.csv'), index=False)
